In [5]:
import random
import copy

# Draughts Endgame : 4 Kings x 2 Kings

In the end I decided to create a game of drafts following an endgame based on the following:

https://lidraughts.org/study/YiAZbWM6

I decided to go with 4 kings vs 2 kings.

In this scenario, there will be only one 'type' of move. Kings are able to move diagonally forward and backwards, as well as capture other pieces.

## Draughts Class
Class which determines start position of pieces, board state, and display the board

In [14]:
class CurrentBoard:

    BOARD_SIZE = 6
    PLAYER_COUNT = 6

    def __init__(self, board=None):
        if board is None:
            self.board = [[" " for _ in range(self.BOARD_SIZE)] for _ in range(self.BOARD_SIZE)]
        else:
            self.board = board
        self.player_positions = {"W": [], "B": []}
        self.state = self.state_of_board()

    def initial_board(self):
        start_positons = random.sample(range(self.BOARD_SIZE*self.BOARD_SIZE), self.PLAYER_COUNT)

        white_kings = start_positons[:2]
        black_kings = start_positons[2:]

        # assign the players their positions on the board by calculating the row and col
        for index in white_kings:
            row = index // self.BOARD_SIZE
            col = index % self.BOARD_SIZE
            self.board[row][col] = "W"
            self.player_positions["W"].append((row,col))
        
        for index in black_kings:
            row = index // self.BOARD_SIZE
            col = index % self.BOARD_SIZE
            self.board[row][col] = "B"
            self.player_positions["B"].append((row,col))


    def display_board(self, game_display = False):
        print("+" + "---+" * self.BOARD_SIZE)
        for row in range(self.BOARD_SIZE):
            print("|", end="")
            for col in range(self.BOARD_SIZE):
                print(f" {self.board[row][col]} |", end="")
            print()
            print("+" + "---+" * self.BOARD_SIZE)

    def state_of_board(self):
        # to check the state of the board for a win, loss, or draw
        state = "Unfinished function"

        # Check win white
        # Check win black
        # Check draw
        return state

    # In this case the only moves are king moves. They can move as outlined above
    def all_possible_moves(self, player_piece):
        possible_moves = []

        player_positions = self.player_positions[player_piece]

        move_directions = [(1, -1), (1, 1), (-1, -1), (-1, 1)] # f-l, f-r, b-l, b-r

        for row, col in player_positions:

            for rowd, cold in move_directions:
                new_row, new_col = row + rowd, col + cold

                for _ in range(self.BOARD_SIZE - 1):

                    if 0 <= new_row < self.BOARD_SIZE and 0 <= new_col < self.BOARD_SIZE:
                        if self.board[new_row][new_col] == " ":
                            new_board = copy.deepcopy(self.board)
                            new_player_positions = copy.deepcopy(self.player_positions)
                            new_board[row][col] = " "
                            new_board[new_row][new_col] = player_piece
                            new_player_positions[player_piece] = [(new_row, new_col)]
                            possible_moves.append(CurrentBoard(new_board))
                        else:
                            break
                        new_row += rowd
                        new_col += cold
                    else:
                        break

        return possible_moves







In [15]:
cb = CurrentBoard()

In [16]:
cb.initial_board()

In [17]:
# check player positions
cb.player_positions

{'W': [(2, 5), (4, 0)], 'B': [(2, 4), (1, 1), (4, 4), (2, 0)]}

In [18]:
# Show initial board
cb.display_board()

+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   | B |   |   |   |   |
+---+---+---+---+---+---+
| B |   |   |   | B | W |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
| W |   |   |   | B |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+


In [19]:
# check all moves fro player based on position
cb.all_possible_moves("W")

In [20]:
cb.all_possible_moves("W")[3].display_board()

+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   | B |   |   | W |   |
+---+---+---+---+---+---+
| B |   |   |   | B |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
| W |   |   |   | B |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+


In [378]:
"""  taken from searchtreeconnect4 :
https://github.com/Robert-Sheehy/AI/blob/main/2024/Search%20Tree/searchtreeconnect4_24.py
"""
class SearchTreeNode:

    def __init__(self,board_instance,playing_as, ply=0):
        self.children = []
        self.max_ply_depth = 3
        self.value_is_assigned = False
        self.ply_depth = ply
        self.current_board = board_instance
        self.move_for = playing_as

        if self.current_board.state == "U":
            if self.ply_depth<= self.max_ply_depth:
                self.generate_children()
            else:
                # evaluation function code
                pass

        else:   # Game over
            if self.current_board.state == "D":
                self.value = 0
            else:
                if ((self.ply_depth % 2) == 0):
                    self.value = -1000
                else:
                    self.value = 1000
            self.value_is_assigned = True

    def min_max_value(self):
        if self.value_is_assigned:
            return self.value

        self.children  = sorted(self.children, key = lambda x:x.min_max_value())

        if ((self.ply_depth % 2) == 0):
            # computers move
            self.value = self.children[-1].value
        else:
            #players move
            self.value = self.children[0].value
        self.value_is_assigned = True

        return self.value

    def generate_children(self):
        for board_for_next_move in self.current_board.all_possible_moves(self.move_for):
            self.children.append(SearchTreeNode(board_for_next_move, self.current_board.other(self.move_for), ply=self.ply_depth + 1))


In [379]:
stn = SearchTreeNode(cb, "W")

In [380]:
stn.children

[]

## PLay Draughts Class

In [26]:
def play_draughts():

    MAX_MOVES = 10

    response = input("Do you want to play first? (y/n)")
    players_turn = (response == "y")

    response = input("Do you want to play as black or white? (b/w)")
    cb = CurrentBoard()

    if response == "b":
        player_is_playing = "B"
    else:
        player_is_playing = "W"

    for _ in range(MAX_MOVES):

        if players_turn:
            cb.display_board(game_display=True)

            while True:
                try:
                    choice = int(input("Make your move (Enter index of board): "))
                    if choice < 0 or choice >= 64:
                        print("Invalid index, please try again (0 and 63)")
                    elif cb.board[choice] != ' ':
                        print("Invalid index, the selected position is occupied")
                    else:
                        break
                except ValueError:
                    print("Invalid input, please enter a number")

            cb = cb.move_piece(choice, player_is_playing)
            cb.display_board()
            
        else:
            search_tree = SearchTreeNode(cb, cb.other(player_is_playing))
            search_tree.min_max_value()
            cb = search_tree.children[-1].current_board
            cb.display_board()

        if cb.state != "U":
            if cb.state == "D":
                print("It's a draw")
            else:
                if players_turn:
                    print("You win")
                else:
                    print("You lose!")

                cb.display_board()
            break

        players_turn = not players_turn


## Play the Game

In [27]:
play_draughts()

+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
|   |   |   |   |   |   |
+---+---+---+---+---+---+
Invalid index, the selected position is occupied
Invalid input, please enter a number
Invalid input, please enter a number
Invalid input, please enter a number
